Get Instances

In [ ]:
SET NOCOUNT ON;

DECLARE @Instances TABLE (
       Value VARCHAR(100),
       InstanceName VARCHAR(100),
       Data VARCHAR(100)
       )

INSERT @Instances
EXECUTE xp_regread
  @rootkey = 'HKEY_LOCAL_MACHINE',
  @key = 'SOFTWARE\Microsoft\Microsoft SQL Server',
  @value_name = 'InstalledInstances' 

-- return your data
SELECT InstanceName FROM @Instances

Get Services

In [ ]:
SET NOCOUNT ON;

select	@@SERVERNAME AS servername
		,DEFAULT_DOMAIN() as domain
		,servicename
		,startup_type_desc
		,status_desc
		,last_startup_time
		,service_account
		,CASE is_clustered
			WHEN 'Y' THEN 1
			WHEN 'N' THEN 0
		END AS is_clustered
		,cluster_nodename
		,CASE instant_file_initialization_enabled
			WHEN 'Y' THEN 1
			WHEN 'N' THEN 0
		END AS instant_file_initialization_enabled
from	sys.dm_server_services;

Get Server Detail and Config

In [ ]:
SET NOCOUNT ON;


DECLARE @ProductVersion VARCHAR(50) = CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR(128))
		,@Version VARCHAR(50) = LEFT(CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR(128)),2)
		,@sql_memory_model_desc VARCHAR(120)
		,@affinity_type_desc VARCHAR(60)
		,@BiosReleaseDate DATE
		,@ProcessorNameString VARCHAR(255);
		
EXEC master.sys.xp_instance_regread
	@rootkey=N'HKEY_LOCAL_MACHINE'
	,@key=N'HARDWARE\DESCRIPTION\System\BIOS'
	,@value_name=N'BiosReleaseDate'
    ,@value=@BiosReleaseDate OUTPUT;
	
EXEC master.sys.xp_instance_regread
	@rootkey=N'HKEY_LOCAL_MACHINE'
	,@key=N'HARDWARE\DESCRIPTION\System\CentralProcessor\0'
	,@value_name=N'ProcessorNameString'
    ,@value=@ProcessorNameString OUTPUT;

IF @Version >= 11
	SELECT	@sql_memory_model_desc = sql_memory_model_desc
			,@affinity_type_desc = affinity_type_desc
	FROM	sys.dm_os_sys_info;

CREATE TABLE #os_sys_info_adds (
	socket_count INT
	,cores_per_socket INT
	,numa_node_count INT
	,softnuma_configuration_desc VARCHAR(60)
);

IF @Version >= 13
	INSERT #os_sys_info_adds (socket_count, cores_per_socket, numa_node_count, softnuma_configuration_desc)
	SELECT	socket_count
			,cores_per_socket
			,numa_node_count
			,softnuma_configuration_desc
	FROM	sys.dm_os_sys_info;

CREATE TABLE #os_windows_info (
	windows_release VARCHAR(256)
	,windows_service_pack_level VARCHAR(256)
	,host_platform VARCHAR(256)
	,host_distribution VARCHAR(256)
);

IF @Version >= 14
	INSERT #os_windows_info (windows_release, windows_service_pack_level, host_platform, host_distribution)
	SELECT	host_release
			,host_service_pack_level
			,host_platform
			,host_distribution
	FROM sys.dm_os_host_info
ELSE
	INSERT #os_windows_info (windows_release, windows_service_pack_level)
	SELECT	windows_release
			, REPLACE(windows_service_pack_level,'Service Pack ','')
	FROM	sys.dm_os_windows_info

SELECT	SERVERPROPERTY('ServerName') AS HostName -- Both the Windows server and instance information associated with a specified instance of SQL Server.
		,SERVERPROPERTY('MachineName') AS MachineName -- For a clustered instance, Windows computer name on which the server instance is running.
		,DEFAULT_DOMAIN() AS Domain
		,ec.local_net_address AS IP
		,ec.local_tcp_port AS Port
		--,SERVERPROPERTY('Edition') AS Product -- Installed product edition of the instance of SQL Server.
		,CASE WHEN @ProductVersion LIKE '8.%' THEN '2000'
			WHEN @ProductVersion LIKE '9.%' THEN '2005'
			WHEN @ProductVersion LIKE '10.0%' THEN '2008'
			WHEN @ProductVersion LIKE '10.5%' THEN '2008 R2'
			WHEN @ProductVersion LIKE '11.%' THEN '2012'
			WHEN @ProductVersion LIKE '12.%' THEN '2014'
			WHEN @ProductVersion LIKE '13.%' THEN '2016'
			WHEN @ProductVersion LIKE '14.%' THEN '2017'
			WHEN @ProductVersion LIKE '15.%' THEN '2019'
		END + ' ' + 
		CASE CAST(SERVERPROPERTY('EngineEdition') AS NVARCHAR(128))
			WHEN '2' THEN 'Std'
			WHEN '3' THEN 'Ent'
			WHEN '4' THEN 'Exp'
		END  +
		CASE WHEN CAST(SERVERPROPERTY('Edition') AS NVARCHAR(128)) LIKE '%64%' THEN ' x64'
			ELSE ' x32'
		END
		AS Product
		,SERVERPROPERTY('ProductLevel') AS SP -- Level of the version of the instance of SQL Server.
		,@ProductVersion AS Version -- Version of the instance of SQL Server, in the form of 'major.minor.build.revision'.
		,ISNULL(SERVERPROPERTY('ProductUpdateLevel'),'') AS CU -- Update level of the current build.
		,CASE WHEN SERVERPROPERTY('IsClustered') = 1 THEN 'Y' ELSE 'N' END AS IsClustered -- Server instance is configured in a failover cluster.
		,CASE WHEN SERVERPROPERTY('IsFullTextInstalled') = 1 THEN 'Y' ELSE 'N' END AS IsFullTextInstalled -- Full-text and semantic indexing components are installed.
		,CASE WHEN SERVERPROPERTY('IsHadrEnabled') = 1 THEN 'Y' ELSE 'N' END AS IsHadrEnabled -- Always On Availability Groups is enabled on this server instance.
		,CAST(ROUND(os.physical_memory_kb / 1024.0 / 1024.0,0) AS INT) AS MemoryGB -- Specifies the total amount of physical memory on the machine. Not nullable. Converted to GB.
		,@sql_memory_model_desc AS MemoryModel
		,mn.memory_nodes AS MemoryNodes
		,@ProcessorNameString AS Processor
		,os.cpu_count AS LogicalCPU
		,tmp_add.socket_count AS SocketCount
		,tmp_add.cores_per_socket AS CoresPerSocket
		,tmp_add.numa_node_count AS NUMANodeCount
		,os.max_workers_count AS MaxWorkersCount
		,@affinity_type_desc AS AffinityType
		,tmp_add.softnuma_configuration_desc AS SoftNUMAConfiguration
		,COALESCE(wi.host_platform
			,CASE WHEN wi.windows_release = '6.1' THEN 'Windows Server 2008 R2'
				WHEN wi.windows_release = '6.2' THEN 'windows Server 2012'
				WHEN wi.windows_release = '6.3' THEN 'windows Server 2012 R2'
				WHEN wi.windows_release = '10.0' THEN 'Windows Server 2016'
				ELSE 'UNKNOWN'
			END -- https://msdn.microsoft.com/library/ms724832%28vs.85%29.aspx?f=255&MSPPError=-2147217396
		) AS OS
		,wi.host_distribution AS OSDescription
		,wi.windows_release AS OSVersion -- For Windows, returns the release number. Cannot be NULL.
		,wi.windows_service_pack_level AS OSServicePack -- For Windows, returns the service pack number. Cannot be NULL.
		,@BiosReleaseDate AS BiosDate
FROM	sys.dm_os_sys_info os
		CROSS JOIN #os_windows_info wi
		CROSS JOIN sys.dm_exec_connections ec
		CROSS JOIN (select COUNT(*) as memory_nodes from sys.dm_os_memory_nodes where memory_node_id <> 64) mn
		CROSS JOIN #os_sys_info_adds tmp_add
WHERE	ec.session_id = @@SPID;

DROP TABLE IF EXISTS #os_sys_info_adds;
DROP TABLE IF EXISTS #os_windows_info;

DECLARE @LoginAuditing INT
		,@DefaultData NVARCHAR(512)
		,@DefaultLog NVARCHAR(512)
		,@DefaultBackup NVARCHAR(512)
		,@NUMANodes INT
		,@TempDbFiles INT
		,@RegKey VARCHAR(100)
		,@SQLVersion VARCHAR(10)
		,@FailSafeOperator VARCHAR(50)
		,@DatabaseMailProfile VARCHAR(50);

EXEC master..xp_instance_regread 
    @rootkey='HKEY_LOCAL_MACHINE',
    @key='SOFTWARE\Microsoft\MSSQLServer\MSSQLServer',
    @value_name='AuditLevel',
    @value=@LoginAuditing OUTPUT;
	
IF @Version >= 11
BEGIN
	SET @DefaultData = CAST(SERVERPROPERTY('InstanceDefaultDataPath') AS VARCHAR(128));
	SET @DefaultLog = CAST(SERVERPROPERTY('InstanceDefaultLogPath') AS VARCHAR(128));
END;
ELSE
BEGIN
	EXEC master.dbo.xp_instance_regread 
		@rootkey='HKEY_LOCAL_MACHINE',
		@key='SOFTWARE\Microsoft\MSSQLServer\MSSQLServer',
		@value_name='DefaultData',
		@value=@DefaultData OUTPUT;

	EXEC master.dbo.xp_instance_regread 
		@rootkey='HKEY_LOCAL_MACHINE',
		@key='SOFTWARE\Microsoft\MSSQLServer\MSSQLServer',
		@value_name='DefaultLog',
		@value=@DefaultLog OUTPUT;
END;

EXEC master.dbo.xp_instance_regread 
    @rootkey='HKEY_LOCAL_MACHINE',
    @key='SOFTWARE\Microsoft\MSSQLServer\MSSQLServer',
    @value_name='BackupDirectory',
    @value=@DefaultBackup OUTPUT;
	
SELECT @SQLVersion = LEFT(CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR(128)),2)
SET @RegKey = REPLACE('SOFTWARE\Microsoft\Microsoft SQL Server\MSSQL00.MSSQLSERVER\SQLServerAgent', '00', @SQLVersion);
EXECUTE master.sys.xp_regread 'HKEY_LOCAL_MACHINE', @RegKey, 'AlertFailSafeOperator', @FailSafeOperator output;
EXECUTE master.sys.xp_regread 'HKEY_LOCAL_MACHINE', @RegKey, 'DatabaseMailProfile', @DatabaseMailProfile output;

/*
http://www.sqlpassion.at/archive/2016/10/17/how-many-numa-nodes-do-i-have/
For every available NUMA node SQL Server creates one dedicated Memory Node (besides Memory Node ID 64, which is always
present for the Dedicated Admin Connection).
*/

SELECT	@TempDbFiles = COUNT(*)
FROM	tempdb.sys.database_files
WHERE	type = 0;

SELECT	CAST(ROUND(MAX(os.physical_memory_kb) / 1024.0 / 1024.0,0) AS INT) AS HostMemory_GB
		,CAST(MAX(CASE WHEN config.name = 'min server memory (MB)' THEN config.value END) AS INT) / 1024 AS MinMemory_GB
		,CAST(MAX(CASE WHEN config.name = 'max server memory (MB)' THEN config.value END) AS INT) / 1024 AS MaxMemory_GB
		,MAX(CASE WHEN SERVERPROPERTY('IsIntegratedSecurityOnly') = 0 THEN 'Y' ELSE 'N' END) AS MixedMode
		,MAX(CASE @LoginAuditing
				WHEN 0 THEN 'None'
				WHEN 1 THEN 'Successful Logins Only'
				WHEN 2 THEN 'Failed Logins Only'
				WHEN 3 THEN 'Both Failed and Successful Logins'
		END) AS LoginAuditing
		,MAX(CASE WHEN config.name = 'cross db ownership chaining' AND config.value = 1 THEN 'Y' ELSE 'N' END) AS CrossDBChaining
		,MAX(CASE WHEN config.name = 'remote access' AND config.value = 1 THEN 'Y' ELSE 'N' END) AS RemoteConnections
		,MAX(CASE WHEN config.name = 'remote proc trans' AND config.value = 1 THEN 'Y' ELSE 'N' END) AS ReqDistTransaction
		,MAX(CASE WHEN config.name = 'fill factor (%)' THEN config.value END) AS DefaultFillFactor
		,MAX(CASE WHEN config.name = 'backup checksum default' AND config.value = 1 THEN 'Y' ELSE 'N' END) AS ChecksumBackup
		,MAX(CASE WHEN config.name = 'backup compression default' AND config.value = 1 THEN 'Y' ELSE 'N' END) AS CompressBackup
		,@DefaultData AS DataPath
		,@DefaultLog AS LogPath
		,@DefaultBackup AS BackupPath
		,MAX(CASE WHEN config.name = 'clr enabled' AND config.value = 1 THEN 'Y' ELSE 'N' END) AS CLREnabled
		,MAX(CASE WHEN config.name = 'clr strict security' AND config.value = 1 THEN 'Y'
				WHEN @Version < 14 THEN ''
				ELSE 'N'
			END) AS CLRStrictSecurity
		,MAX(CASE WHEN config.name = 'filestream access level' THEN CASE CAST(config.value AS INT)
				WHEN 0 THEN 'Disabled'
				WHEN 1 THEN 'Transact-SQL access enabled'
				WHEN 2 THEN 'Full access enabled'
		END END) AS FILESTREAMAccessLevel
		,MAX(CASE WHEN config.name = 'server trigger recursion' AND config.value = 1 THEN 'Y' ELSE 'N' END) AS RecursiveTriggers
		,MAX(CASE WHEN config.name = 'optimize for ad hoc workloads' AND config.value = 1 THEN 'Y' ELSE 'N' END) AS OptimizeForAdHocWorkloads
		,MAX(CASE WHEN config.name = 'scan for startup procs' AND config.value = 1 THEN 'Y' ELSE 'N' END) AS ScanForStartupProcs
		,MAX(CASE WHEN config.name = 'cost threshold for parallelism' THEN config.value END) AS CostThresholdParallelism
		,MAX(os.cpu_count) AS HostCPUs
		,MAX(CASE WHEN config.name = 'max degree of parallelism' THEN config.value END) AS MAXDOP
		,@TempDbFiles AS TempDbFiles
		,MAX(CASE WHEN config.name = 'Database Mail XPs' AND config.value = 1 THEN 'Y' ELSE 'N' END) AS DBMailEnabled
		,MAX(CASE WHEN config.name = 'remote admin connections' AND config.value = 1 THEN 'Y' ELSE 'N' END) AS DACEnabled
		,@FailSafeOperator as FailsafeOperator
		,@DatabaseMailProfile as DatabaseMailProfile
		,CAST(SERVERPROPERTY('Collation') AS NVARCHAR(128)) AS Collation
FROM	sys.configurations config
		CROSS JOIN sys.dm_os_sys_info os;

---- http://blog.waynesheffield.com/wayne/archive/2017/09/registry-sql-server-startup-parameters/

--DECLARE @RegHive    VARCHAR(50),
--        @RegKey     VARCHAR(100);
 
--SET @RegHive = 'HKEY_LOCAL_MACHINE';
--SET @RegKey  = 'Software\Microsoft\MSSQLSERVER\MSSQLServer\Parameters';
 
---- Get all of the arguments / parameters when starting up the service.
--DECLARE @SQLArgs TABLE (
--    Value   VARCHAR(50),
--    Data    VARCHAR(500),
--    ArgNum  AS CONVERT(INTEGER, REPLACE(Value, 'SQLArg', '')));
 
--INSERT INTO @SQLArgs
--EXECUTE master.sys.xp_instance_regenumvalues @RegHive, @RegKey;
 
--SELECT  Value AS StartupParam
--       ,Data AS StartupData
--FROM    @SQLArgs;

SELECT	value_name AS StartupParam
		,value_data AS StartupData
FROM	sys.dm_server_registry  
WHERE	registry_key LIKE N'%Parameters';  

DBCC TRACESTATUS;



Get DatabaseDetail and Config

In [ ]:
SET NOCOUNT ON;


DECLARE @tmp_ExtProperties TABLE (DatabaseName VARCHAR(128), name NVARCHAR(60), value SQL_VARIANT);

INSERT INTO @tmp_ExtProperties (DatabaseName, name, value)
EXEC sp_MSForEachDB 'Use [?];
SELECT DB_NAME() AS DatabaseName, name, value FROM sys.extended_properties WHERE name = ''Confluence Description''';

DECLARE @tmp_FileStream TABLE (DatabaseName VARCHAR(128), non_transacted_access TINYINT, directory_name NVARCHAR(255));

INSERT @tmp_FileStream (DatabaseName, non_transacted_access, directory_name)
EXEC sp_MSForEachDB 'Use [?];
SELECT DB_NAME() AS DatabaseName, non_transacted_access, directory_name FROM sys.database_filestream_options';

DECLARE @tmp_QueryStore TABLE (
	DatabaseName VARCHAR(128)
	,querystore_desired_state SMALLINT
	,querystore_actual_state SMALLINT
	,querystore_readonly_reason INT
	,querystore_current_storage_size_mb BIGINT
	,querystore_flush_interval_seconds BIGINT
	,querystore_interval_length_minutes BIGINT
	,querystore_max_storage_size_mb BIGINT
	,querystore_stale_query_threshold_days BIGINT
	,querystore_max_plans_per_query BIGINT
	,querystore_query_capture_mode SMALLINT
	,querystore_size_based_cleanup_mode SMALLINT
	,querystore_wait_stats_capture_mode SMALLINT
);

DECLARE @QS_SQL NVARCHAR(MAX)
		,@Version VARCHAR(50) = LEFT(CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR(128)),2);

IF @Version > 12
BEGIN
SET @QS_SQL = 'Use [?];
SELECT DB_NAME() AS DatabaseName
	,desired_state
	,actual_state
	,readonly_reason
	,current_storage_size_mb
	,flush_interval_seconds
	,interval_length_minutes
	,max_storage_size_mb
	,stale_query_threshold_days
	,max_plans_per_query
	,query_capture_mode
	,size_based_cleanup_mode'
+ CASE WHEN @Version > 13 THEN '	,wait_stats_capture_mode' ELSE '	,NULL' END + '
FROM sys.database_query_store_options';

INSERT @tmp_QueryStore (
	DatabaseName
	,querystore_desired_state
	,querystore_actual_state
	,querystore_readonly_reason
	,querystore_current_storage_size_mb
	,querystore_flush_interval_seconds
	,querystore_interval_length_minutes
	,querystore_max_storage_size_mb
	,querystore_stale_query_threshold_days
	,querystore_max_plans_per_query
	,querystore_query_capture_mode
	,querystore_size_based_cleanup_mode
	,querystore_wait_stats_capture_mode
)
EXEC sp_MSForEachDB @QS_SQL;
END;

SELECT *
INTO #databases
FROM sys.databases;

IF @Version < 12
BEGIN
	ALTER TABLE #databases ADD
		is_auto_create_stats_incremental_on BIT
		,delayed_durability INT
		,is_query_store_on BIT;
END;

IF @Version < 13
BEGIN
	ALTER TABLE #databases ADD
		is_mixed_page_allocation_on BIT;;
END;

CREATE TABLE #dm_hadr_database_replica_states (
	database_id INT
	,group_id UNIQUEIDENTIFIER
	,is_primary_replica BIT
);

IF @Version > 12
	INSERT #dm_hadr_database_replica_states (database_id, group_id, is_primary_replica)
	SELECT	c.database_id
			,b.group_id
			,sys.fn_hadr_is_primary_replica (DB_Name(c.database_id)) AS is_primary_replica
	FROM	sys.availability_replicas AS b
			JOIN sys.dm_hadr_database_replica_states AS c ON b.replica_id = c.replica_id
	WHERE	b.replica_server_name = @@SERVERNAME;
ELSE
	INSERT #dm_hadr_database_replica_states (database_id, group_id, is_primary_replica)
	SELECT c.database_id, a.group_id
		,CASE a.role_desc
			WHEN 'PRIMARY' THEN 1
			WHEN 'SECONDARY' THEN 0
			WHEN 'RESOLVING' THEN 0
			ELSE NULL
		END AS is_primary_replica
	FROM sys.dm_hadr_availability_replica_states AS a
		JOIN sys.availability_replicas AS b ON b.replica_id = a.replica_id
		JOIN sys.dm_hadr_database_replica_states AS c ON a.replica_id = c.replica_id
	WHERE b.replica_server_name = @@SERVERNAME;

SELECT ag_id, ag_name
INTO #dm_hadr_name_id_map
FROM sys.dm_hadr_name_id_map;

SELECT sid, name
INTO #server_principals
FROM sys.server_principals;

select distinct
	@@SERVERNAME as servername
	,sd.name as databasename
	,grp.ag_name as agname
	,hdrs.is_primary_replica
	,ep.value as description
	,ISNULL(u.name, '') as databaseowner
	,sd.collation_name as collation
	,sd.recovery_model_desc as recoverymodel
	,sd.compatibility_level as compatibilitylevel
	,sd.containment_desc as containmenttype
	,sd.is_auto_close_on
	,sd.is_auto_create_stats_incremental_on
	,sd.is_auto_create_stats_on
	,sd.is_auto_shrink_on
	,sd.is_auto_update_stats_on
	,sd.is_auto_update_stats_async_on
	,sd.is_cursor_close_on_commit_on
	,sd.is_local_cursor_default
	,sd.snapshot_isolation_state
	,sd.is_ansi_null_default_on
	,sd.is_ansi_nulls_on
	,sd.is_ansi_padding_on
	,sd.is_ansi_warnings_on
	,sd.is_arithabort_on
	,sd.is_concat_null_yields_null_on
	,sd.is_db_chaining_on
	,sd.is_date_correlation_on
	,sd.delayed_durability
	,sd.is_mixed_page_allocation_on
	,sd.is_read_committed_snapshot_on
	,sd.is_numeric_roundabort_on
	,sd.is_parameterization_forced
	,sd.is_quoted_identifier_on
	,sd.is_recursive_triggers_on
	,sd.is_nested_triggers_on
	,sd.is_trustworthy_on
	,sd.page_verify_option
	,sd.target_recovery_time_in_seconds
	,sd.is_broker_enabled
	,sd.is_honor_broker_priority_on
	,sd.service_broker_guid
	,sd.is_read_only
	,sd.state
	,sd.is_in_standby
	,sd.is_encrypted
	,sd.is_master_key_encrypted_by_server
	,sd.user_access
	,sd.is_cdc_enabled
	,sd.is_fulltext_enabled
	,fs.non_transacted_access
	,fs.directory_name
	,sd.is_query_store_on
	,qs.querystore_desired_state
	,qs.querystore_actual_state
	,qs.querystore_readonly_reason
	,qs.querystore_current_storage_size_mb
	,qs.querystore_flush_interval_seconds
	,qs.querystore_interval_length_minutes
	,qs.querystore_max_storage_size_mb
	,qs.querystore_stale_query_threshold_days
	,qs.querystore_max_plans_per_query
	,qs.querystore_query_capture_mode
	,qs.querystore_size_based_cleanup_mode
	,qs.querystore_wait_stats_capture_mode
from #databases as sd
    left outer join #dm_hadr_database_replica_states as hdrs on hdrs.database_id = sd.database_id
    left outer join #dm_hadr_name_id_map as grp on grp.ag_id = hdrs.group_id
    left outer join @tmp_ExtProperties ep on ep.DatabaseName = sd.name
	left outer join #server_principals u on sd.owner_sid = u.sid
	left outer join @tmp_FileStream fs on fs.DatabaseName = sd.name
	left outer join @tmp_QueryStore qs on qs.DatabaseName = sd.name
where sd.name not in ('master','msdb','model','tempdb', 'ReportServer', 'ReportServerTempDB');

IF OBJECT_ID('tempdb..#databases') IS NOT NULL
 DROP TABLE #databases;

IF OBJECT_ID('tempdb..#dm_hadr_database_replica_states') IS NOT NULL
 DROP TABLE #dm_hadr_database_replica_states;
 
IF OBJECT_ID('tempdb..#dm_hadr_name_id_map') IS NOT NULL
 DROP TABLE #dm_hadr_name_id_map;

IF OBJECT_ID('tempdb..#server_principals') IS NOT NULL
 DROP TABLE #server_principals;


Get Database Scoped Config

In [ ]:
SET NOCOUNT ON;

DECLARE @Version VARCHAR(50) = LEFT(CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR(128)),2)
		,@SQL NVARCHAR(MAX);
		
IF @Version = 13
	SET @SQL = N'
	SELECT	@@SERVERNAME as servername
		    ,DB_NAME() as dbname
			,name
			,value
			,value_for_secondary
			,NULL AS is_value_default
	FROM	sys.database_scoped_configurations;'
ELSE IF @Version > 13
	SET @SQL = N'
	SELECT	@@SERVERNAME as servername
		    ,DB_NAME() as dbname
			,name
			,value
			,value_for_secondary
			,is_value_default
	FROM	sys.database_scoped_configurations;'

EXEC (@SQL);

Get Database File Detail

In [ ]:
SET NOCOUNT ON;


declare @sql_used nvarchar(max) = N'USE [?]; INSERT #DbUsed (DatabaseId, Used, Name) SELECT DB_ID(), CONVERT(BIGINT, FILEPROPERTY(name, ''SpaceUsed'')) * 8 / 1024, name from sys.database_files'
		,@sql_log nvarchar(max) = N'USE [?]; INSERT #LogInfo (RecoveryUnitId, FileId, FileSize, StartOffset, FSeqNo, [Status], Parity, CreateLSN) EXEC(''DBCC LOGINFO'')'
		,@Version VARCHAR(50) = LEFT(CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR(128)),2);

if OBJECT_ID('tempdb..#DbUsed') is not null drop table #DbUsed;
create table #DbUsed (DatabaseId INT, Used BIGINT, name VARCHAR(128));

if OBJECT_ID('tempdb..#LogInfo') is not null drop table #LogInfo;
create table #LogInfo (DatabaseName NVARCHAR(128) default DB_NAME(), RecoveryUnitId INT, FileId INT, FileSize BIGINT, StartOffset BIGINT, FSeqNo INT, [Status] TINYINT, Parity TINYINT, CreateLSN NUMERIC(25,0));

if OBJECT_ID('tempdb..#dm_hadr_database_replica_states') is not null drop table #dm_hadr_database_replica_states;
CREATE TABLE #dm_hadr_database_replica_states (database_id INT, is_primary_replica BIT);

exec sp_msforeachdb @sql_used;
exec sp_msforeachdb @sql_log;

IF @Version > 12
	INSERT #dm_hadr_database_replica_states (database_id, is_primary_replica)
	SELECT	c.database_id
			,sys.fn_hadr_is_primary_replica (DB_Name(c.database_id)) AS is_primary_replica
	FROM	sys.availability_replicas AS b
			JOIN sys.dm_hadr_database_replica_states AS c ON b.replica_id = c.replica_id
	WHERE	b.replica_server_name = @@SERVERNAME;
ELSE
	INSERT #dm_hadr_database_replica_states (database_id, is_primary_replica)
	SELECT c.database_id
		,CASE a.role_desc
			WHEN 'PRIMARY' THEN 1
			WHEN 'SECONDARY' THEN 0
			WHEN 'RESOLVING' THEN 0
			ELSE NULL
		END AS is_primary_replica
	FROM sys.dm_hadr_availability_replica_states AS a
		JOIN sys.availability_replicas AS b ON b.replica_id = a.replica_id
		JOIN sys.dm_hadr_database_replica_states AS c ON a.replica_id = c.replica_id
	WHERE b.replica_server_name = @@SERVERNAME;

SELECT @@SERVERNAME as servername
		, DB_NAME(f.database_id) as databasename
		, UPPER(LEFT(f.physical_name, 3)) as volume
		, f.name
		, f.physical_name
		, CASE WHEN DB_NAME(f.database_id) = 'tempdb' THEN CONVERT(NVARCHAR(15), CONVERT(BIGINT, tmpdb.size) * 8 / 1024)
			ELSE CONVERT(NVARCHAR(15), CONVERT(BIGINT, f.size) * 8 / 1024)
		END as filesize_mb
		, tmp.Used
		, CASE f.max_size WHEN -1 THEN N'Unlimited' ELSE CONVERT(NVARCHAR(15), CONVERT(BIGINT, f.max_size) * 8 / 1024) END as maxfilesize_mb
		, CASE f.is_percent_growth WHEN 1 THEN NULL ELSE (f.growth * 8 / 1024) END as filegrowth_mb
		, CASE f.is_percent_growth WHEN 1 THEN f.growth ELSE NULL END as filegrowth_pct
		, li.VLFs
		, rs.is_primary_replica
		, DEFAULT_DOMAIN() as domain
FROM	sys.master_files f
		LEFT OUTER JOIN #DbUsed tmp on f.database_id = tmp.DatabaseId and f.name = tmp.name
		LEFT OUTER JOIN (select DatabaseName, COUNT(FileId) as VLFs from #LogInfo group by DatabaseName) li on DB_NAME(f.database_id) = li.DatabaseName and f.name like '%log'
		LEFT OUTER JOIN #dm_hadr_database_replica_states rs on f.database_id = rs.database_id
		LEFT OUTER JOIN tempdb.sys.database_files tmpdb ON f.name = tmpdb.name AND DB_NAME(f.database_id) = 'tempdb';

Get Index Details

In [ ]:
SET NOCOUNT ON;

DECLARE @sql_indxinfo NVARCHAR(2000)
		,@sql_indxsize NVARCHAR(2000)
		,@Version VARCHAR(50) = LEFT(CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR(128)),2)
		,@IsHadrEnabled TINYINT = CONVERT(TINYINT,SERVERPROPERTY ('IsHadrEnabled'))
		,@RoleDesc NVARCHAR(60) = 'PRIMARY';

if OBJECT_ID('tempdb..#IndxInfo') is not null drop table #IndxInfo;
create table #IndxInfo (
	databaseid INT NOT NULL
	,objectid INT NOT NULL
	,indexid INT NOT NULL
	,databasename VARCHAR(128) NOT NULL
	,schemaname VARCHAR(128) NOT NULL
	,tablename VARCHAR(128) NOT NULL
	,indexname VARCHAR(128) NULL
	,indextype TINYINT NOT NULL
	,columns VARCHAR(2000) NULL
	,included_columns VARCHAR(2000) NULL
	,is_unique BIT NOT NULL
	,is_primary_key BIT NOT NULL
	,is_unique_constraint BIT NOT NULL
	,is_disabled BIT NOT NULL
	,has_filter BIT NOT NULL
	,filter_definition VARCHAR(2000) NULL
	,auto_created BIT NULL
	,fill_factor TINYINT NOT NULL
	,is_padded BIT NOT NULL
	,last_user_seek DATETIME NULL
	,last_user_scan DATETIME NULL
	,last_user_lookup DATETIME NULL
	,last_user_update DATETIME NULL
	,user_seeks BIGINT NULL
	,user_scans BIGINT NULL
	,user_lookups BIGINT NULL
	,user_updates BIGINT NULL
);

if OBJECT_ID('tempdb..#IndxSize') is not null drop table #IndxSize;
create table #IndxSize (
	databaseid INT NOT NULL
	,objectid INT NOT NULL
	,indexid INT NOT NULL
	,partitions INT NOT NULL
	,rows BIGINT NOT NULL
	,total_pages BIGINT NOT NULL
	,used_pages BIGINT NOT NULL
	,in_row_data_pages BIGINT NOT NULL
	,lob_data_pages BIGINT NOT NULL
	,row_overflow_data_pages BIGINT NOT NULL
	,data_compression TINYINT NOT NULL
);

SET @sql_indxinfo = N'USE [?];
IF DB_ID() > 4
INSERT #IndxInfo
SELECT	DB_ID()
		,i.object_id
		,i.index_id
		,DB_NAME()
		,SCHEMA_NAME(o.schema_id)
		,o.name
		,i.name
		,i.type
       ,STUFF((
            SELECT	CASE WHEN ic.is_descending_key = 1 THEN '', '' + c.name + ''(-)'' ELSE '', '' + c.name END
            FROM	sys.index_columns ic
					JOIN sys.columns c
						ON  c.object_id = ic.object_id
						AND c.column_id = ic.column_id
            WHERE	ic.object_id = i.object_id
            AND		ic.index_id = i.index_id
            AND		ic.is_included_column = 0
            ORDER BY ic.key_ordinal
            FOR XML PATH('''')
        ), 1, 2, '''')
       ,STUFF((
            SELECT	CASE WHEN ic.is_descending_key = 1 THEN '', '' + c.name + ''(-)'' ELSE '', '' + c.name END
            FROM	sys.index_columns ic
					JOIN sys.columns c
						ON  c.object_id = ic.object_id
						AND c.column_id = ic.column_id
            WHERE	ic.object_id = i.object_id
            AND		ic.index_id = i.index_id
            AND		ic.is_included_column = 1
            ORDER BY ic.key_ordinal
            FOR XML PATH('''')
        ), 1, 2, '''')
		,i.is_unique
		,i.is_primary_key
		,i.is_unique_constraint
		,i.is_disabled
		,i.has_filter
		,i.filter_definition'
+ CASE WHEN @Version > 13 THEN '		,i.auto_created' ELSE '		,NULL' END +
'		,i.fill_factor
		,i.is_padded
		,u.last_user_seek
		,u.last_user_scan
		,u.last_user_lookup
		,u.last_user_update
		,u.user_seeks
		,u.user_scans
		,u.user_lookups
		,u.user_updates
FROM	sys.objects o
		JOIN sys.indexes i ON o.object_id = i.object_id
		LEFT JOIN sys.dm_db_index_usage_stats u ON i.object_id = u.object_id and i.index_id = u.index_id and u.database_id = DB_ID()
WHERE	OBJECTPROPERTY(o.object_id,''IsUserTable'') = 1;'

SET @sql_indxsize = N'USE [?];
IF DB_ID() > 4
INSERT #IndxSize
SELECT	DB_ID()
		,i.object_id
		,i.index_id
		,COUNT(p.partition_id)
		,SUM(p.rows)
		,SUM(a.used_pages)
		,SUM(a.total_pages)
		,SUM(a.in_row)
		,SUM(a.lob)
		,SUM(a.row_overflow)
		,p.data_compression
FROM	sys.indexes i
		JOIN sys.partitions p ON i.object_id = p.object_id AND i.index_id = p.index_id
		JOIN (
			SELECT	container_id
					,SUM(used_pages) as used_pages
					,SUM(total_pages) as total_pages
					,SUM(CASE WHEN type = 1 THEN data_pages ELSE 0 END) as in_row
					,SUM(CASE WHEN type = 2 THEN data_pages ELSE 0 END) as lob
					,SUM(CASE WHEN type = 3 THEN data_pages ELSE 0 END) as row_overflow
			FROM	sys.allocation_units
			GROUP BY container_id
		) a ON p.partition_id = a.container_id
WHERE	OBJECTPROPERTY(i.object_id,''IsUserTable'') = 1
GROUP BY i.object_id
		,i.index_id
		,p.data_compression;'
		
IF @IsHadrEnabled = 0
OR (
	@IsHadrEnabled = 1
	AND EXISTS (
		SELECT 1
		FROM sys.dm_hadr_availability_replica_states AS a
			JOIN sys.availability_replicas AS b
		ON b.replica_id = a.replica_id
		WHERE b.replica_server_name = @@SERVERNAME
		AND	a.role_desc = @RoleDesc
	)
)
BEGIN
	EXEC sp_msforeachdb @sql_indxinfo;
	EXEC sp_msforeachdb @sql_indxsize;
END;

SELECT	@@SERVERNAME AS servername
		,i.databasename
		,i.schemaname
		,i.tablename
		,i.indexname
		,i.indextype
		,i.columns
		,i.included_columns
		,i.is_unique
		,i.is_primary_key
		,i.is_unique_constraint
		,i.is_disabled
		,i.has_filter
		,i.filter_definition
		,i.auto_created
		,i.fill_factor
		,i.is_padded
		,s.partitions
		,s.rows
		,s.total_pages
		,s.used_pages
		,s.in_row_data_pages
		,s.lob_data_pages
		,s.row_overflow_data_pages
		,s.data_compression
		,i.last_user_seek
		,i.last_user_scan
		,i.last_user_lookup
		,i.last_user_update
		,i.user_seeks
		,i.user_scans
		,i.user_lookups
		,i.user_updates
FROM	#IndxInfo i
		JOIN #IndxSize s
			ON s.databaseid = i.databaseid
			AND s.objectid = i.objectid
			AND s.indexid = i.indexid
WHERE	i.databaseid > 4;


Get SQL Agent Alerts

In [ ]:
SET NOCOUNT ON;

SELECT @@SERVERNAME AS servername 
	,name 
	,message_id 
	,severity 
	,enabled 
	,delay_between_responses 
	,include_event_description 
	,has_notification
FROM msdb.dbo.sysalerts;


Get SQL Agent Jobs

In [ ]:
SET NOCOUNT ON;

-- Query to display schedules of jobs.  
--      copied from http://www.sqlservercentral.com/scripts/Maintenance+and+Management/62222/
--               posted there by "Cowboy DBA"
--    ----------

DECLARE @LastModDate DATETIME --= '11/1/2016'
		, @Enabled BIT --= 1; -- NULL returns both Enabled & Disabled jobs

SELECT	SERVERPROPERTY ('ServerName') as [Server]
		, msdb.dbo.sysjobs.name as [Job]
		, [JobSteps] = stepcount
		, [Enabled] = CASE msdb.dbo.sysjobs.[enabled] WHEN 1 THEN 'Y' ELSE 'N' END 
		, [StartStepId] = msdb.dbo.sysjobs.start_step_id
		, [Schedule] =
			CASE WHEN msdb.dbo.sysjobs.[enabled] = 0
					THEN 'Disabled'
				WHEN msdb.dbo.sysjobs.job_id IS NULL
					THEN 'Unscheduled'
				WHEN msdb.dbo.sysschedules.freq_type = 0x1 -- OneTime
					THEN 'Once on ' + CONVERT(CHAR(10), CAST(CAST(msdb.dbo.sysschedules.active_start_date AS VARCHAR)AS DATETIME), 101)
				WHEN msdb.dbo.sysschedules.freq_type = 0x4 -- Daily
					THEN 'Daily'
				WHEN msdb.dbo.sysschedules.freq_type = 0x8 -- weekly
					THEN CASE WHEN msdb.dbo.sysschedules.freq_recurrence_factor = 1
								THEN 'Wkly on '
							WHEN msdb.dbo.sysschedules.freq_recurrence_factor > 1
								THEN 'Every ' + CAST(msdb.dbo.sysschedules.freq_recurrence_factor AS VARCHAR) + ' wks on '
						END
						+ LEFT(
							CASE WHEN msdb.dbo.sysschedules.freq_interval &  1 =  1 THEN 'Su, ' ELSE '' END
							+ CASE WHEN msdb.dbo.sysschedules.freq_interval &  2 =  2 THEN 'M, ' ELSE '' END
							+ CASE WHEN msdb.dbo.sysschedules.freq_interval &  4 =  4 THEN 'Tu, ' ELSE '' END
							+ CASE WHEN msdb.dbo.sysschedules.freq_interval &  8 =  8 THEN 'W, ' ELSE '' END
							+ CASE WHEN msdb.dbo.sysschedules.freq_interval & 16 = 16 THEN 'Th, ' ELSE '' END
							+ CASE WHEN msdb.dbo.sysschedules.freq_interval & 32 = 32 THEN 'F, ' ELSE '' END
							+ CASE WHEN msdb.dbo.sysschedules.freq_interval & 64 = 64 THEN 'Sa, ' ELSE '' END
							, LEN(
								CASE WHEN msdb.dbo.sysschedules.freq_interval &  1 =  1 THEN 'Su, ' ELSE '' END
								+ CASE WHEN msdb.dbo.sysschedules.freq_interval &  2 =  2 THEN 'M, ' ELSE '' END
								+ CASE WHEN msdb.dbo.sysschedules.freq_interval &  4 =  4 THEN 'Tu, ' ELSE '' END
								+ CASE WHEN msdb.dbo.sysschedules.freq_interval &  8 =  8 THEN 'W, ' ELSE '' END
								+ CASE WHEN msdb.dbo.sysschedules.freq_interval & 16 = 16 THEN 'Th, ' ELSE '' END
								+ CASE WHEN msdb.dbo.sysschedules.freq_interval & 32 = 32 THEN 'F, ' ELSE '' END
								+ CASE WHEN msdb.dbo.sysschedules.freq_interval & 64 = 64 THEN 'Sa, ' ELSE '' END
							)  - 1  -- LEN() ignores trailing spaces
						)
				WHEN msdb.dbo.sysschedules.freq_type = 0x10 -- monthly
					THEN CASE WHEN msdb.dbo.sysschedules.freq_recurrence_factor = 1
									THEN 'Mthly on the '
								WHEN msdb.dbo.sysschedules.freq_recurrence_factor > 1
								   THEN 'Every ' + CAST(msdb.dbo.sysschedules.freq_recurrence_factor AS VARCHAR) + ' mo on the '
							END
						+ CAST(msdb.dbo.sysschedules.freq_interval AS VARCHAR)
						+ CASE WHEN msdb.dbo.sysschedules.freq_interval IN (1, 21, 31)
									THEN 'st'
								WHEN msdb.dbo.sysschedules.freq_interval IN (2, 22)
									THEN 'nd'
								WHEN msdb.dbo.sysschedules.freq_interval IN (3, 23)
									THEN 'rd'
								ELSE 'th'
							END
				WHEN msdb.dbo.sysschedules.freq_type = 0x20 -- monthly relative
						THEN CASE WHEN msdb.dbo.sysschedules.freq_recurrence_factor = 1
										THEN 'Mthly on the '
									WHEN msdb.dbo.sysschedules.freq_recurrence_factor > 1
										THEN 'Every '+ CAST( msdb.dbo.sysschedules.freq_recurrence_factor AS VARCHAR )+ ' mths on the '
								END
						+ CASE msdb.dbo.sysschedules.freq_relative_interval
								WHEN 0x01 THEN 'first '
								WHEN 0x02 THEN 'second '
								WHEN 0x04 THEN 'third '
								WHEN 0x08 THEN 'fourth '
								WHEN 0x10 THEN 'last '
							END
						+ CASE msdb.dbo.sysschedules.freq_interval
								WHEN  1 THEN 'Su'
								WHEN  2 THEN 'M'
								WHEN  3 THEN 'Tu'
								WHEN  4 THEN 'W'
								WHEN  5 THEN 'Th'
								WHEN  6 THEN 'F'
								WHEN  7 THEN 'Sa'
								WHEN  8 THEN 'day'
								WHEN  9 THEN 'wk day'
								WHEN 10 THEN 'wknd day'
							END
				WHEN msdb.dbo.sysschedules.freq_type = 0x40
				   THEN 'Auto start when SQLServerAgent starts'
				WHEN msdb.dbo.sysschedules.freq_type = 0x80
				   THEN 'Starts whenever CPUs become idle'
				ELSE ''
			END
		+ CASE WHEN msdb.dbo.sysjobs.[enabled] = 0
					THEN ''
				WHEN msdb.dbo.sysjobs.job_id IS NULL
					THEN ''
				WHEN msdb.dbo.sysschedules.freq_subday_type = 0x1
				OR msdb.dbo.sysschedules.freq_type = 0x1
					THEN ' at '
						+ CASE -- Depends on time being integer to drop right-side digits
							WHEN(msdb.dbo.sysschedules.active_start_time % 1000000)/10000 = 0
								THEN '12'
									+ ':'  
									+ REPLICATE('0',2 - LEN(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100)))
									+ CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100) 
									+ ' AM'
							WHEN (msdb.dbo.sysschedules.active_start_time % 1000000)/10000< 10
								THEN CONVERT(CHAR(1),(msdb.dbo.sysschedules.active_start_time % 1000000)/10000) 
									+ ':'  
									+ REPLICATE('0',2 - LEN(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100))) 
									+ CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100) 
									+ ' AM'
							WHEN (msdb.dbo.sysschedules.active_start_time % 1000000)/10000 < 12
								THEN CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 1000000)/10000) 
									+ ':'  
									+ REPLICATE('0',2 - LEN(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100))) 
									+ CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100) 
									+ ' AM'
							WHEN (msdb.dbo.sysschedules.active_start_time % 1000000)/10000< 22
								THEN CONVERT(CHAR(1),((msdb.dbo.sysschedules.active_start_time % 1000000)/10000) - 12) 
									+ ':'  
									+ REPLICATE('0',2 - LEN(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100))) 
									+ CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100) 
									+ ' PM'
							ELSE CONVERT(CHAR(2),((msdb.dbo.sysschedules.active_start_time % 1000000)/10000) - 12)
								+ ':'  
								+ REPLICATE('0',2 - LEN(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100))) 
								+ CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100) 
								+ ' PM'
						END
				WHEN msdb.dbo.sysschedules.freq_subday_type IN (0x2, 0x4, 0x8)
					THEN ' every '
						+ CAST(msdb.dbo.sysschedules.freq_subday_interval AS VARCHAR)
						+ CASE freq_subday_type
							WHEN 0x2
								THEN ' sec'
							WHEN 0x4
								THEN ' min'
							WHEN 0x8
								THEN ' hr'
						END
					+ CASE
						WHEN msdb.dbo.sysschedules.freq_subday_interval > 1
							THEN 's'
						ELSE '' -- Added default 3/21/08; John Arnott
					END
				ELSE ''
		END
		+ CASE WHEN msdb.dbo.sysjobs.[enabled] = 0
					THEN ''
				WHEN msdb.dbo.sysjobs.job_id IS NULL
					THEN ''
				WHEN msdb.dbo.sysschedules.freq_subday_type IN (0x2, 0x4, 0x8)
					THEN ' btw '
						+ CASE -- Depends on time being integer to drop right-side digits
							WHEN(msdb.dbo.sysschedules.active_start_time % 1000000)/10000 = 0
								THEN '12'
								+ ':'  
								+ REPLICATE('0',2 - LEN(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100)))
								+ RTRIM(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100))
								+ ' AM'
							WHEN (msdb.dbo.sysschedules.active_start_time % 1000000)/10000< 10
								THEN CONVERT(CHAR(1),(msdb.dbo.sysschedules.active_start_time % 1000000)/10000) 
									+ ':'  
									+ REPLICATE('0',2 - LEN(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100))) 
									+ RTRIM(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100))
									+ ' AM'
							WHEN (msdb.dbo.sysschedules.active_start_time % 1000000)/10000 < 12
								THEN CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 1000000)/10000) 
									+ ':'  
									+ REPLICATE('0',2 - LEN(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100))) 
									+ RTRIM(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100)) 
									+ ' AM'
							WHEN (msdb.dbo.sysschedules.active_start_time % 1000000)/10000< 22
								THEN CONVERT(CHAR(1),((msdb.dbo.sysschedules.active_start_time % 1000000)/10000) - 12) 
									+ ':'  
									+ REPLICATE('0',2 - LEN(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100))) 
									+ RTRIM(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100)) 
									+ ' PM'
							ELSE CONVERT(CHAR(2),((msdb.dbo.sysschedules.active_start_time % 1000000)/10000) - 12)
								+ ':'  
								+ REPLICATE('0',2 - LEN(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100))) 
								+ RTRIM(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_start_time % 10000)/100))
								+ ' PM'
						END
						+ ' and '
						+ CASE -- Depends on time being integer to drop right-side digits
							WHEN (msdb.dbo.sysschedules.active_end_time % 1000000)/10000 = 0
								THEN '12'
									+ ':'  
									+ REPLICATE('0',2 - LEN(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_end_time % 10000)/100)))
									+ RTRIM(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_end_time % 10000)/100))
									+ ' AM'
							WHEN (msdb.dbo.sysschedules.active_end_time % 1000000)/10000< 10
								THEN CONVERT(CHAR(1),(msdb.dbo.sysschedules.active_end_time % 1000000)/10000) 
								+ ':'  
								+ REPLICATE('0',2 - LEN(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_end_time % 10000)/100))) 
								+ RTRIM(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_end_time % 10000)/100))
								+ ' AM'
							WHEN (msdb.dbo.sysschedules.active_end_time % 1000000)/10000 < 12
								THEN CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_end_time % 1000000)/10000) 
								+ ':'  
								+ REPLICATE('0',2 - LEN(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_end_time % 10000)/100))) 
								+ RTRIM(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_end_time % 10000)/100))
								+ ' AM'
							WHEN (msdb.dbo.sysschedules.active_end_time % 1000000)/10000< 22
								THEN CONVERT(CHAR(1),((msdb.dbo.sysschedules.active_end_time % 1000000)/10000) - 12)
								+ ':'  
								+ REPLICATE('0',2 - LEN(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_end_time % 10000)/100))) 
								+ RTRIM(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_end_time % 10000)/100)) 
								+ ' PM'
							ELSE CONVERT(CHAR(2),((msdb.dbo.sysschedules.active_end_time % 1000000)/10000) - 12)
								+ ':'  
								+ REPLICATE('0',2 - LEN(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_end_time % 10000)/100))) 
								+ RTRIM(CONVERT(CHAR(2),(msdb.dbo.sysschedules.active_end_time % 10000)/100)) 
								+ ' PM'
						END
				ELSE ''
		END
		, [Notify_Level_Eventlog] =
		CASE msdb.dbo.sysjobs.notify_level_eventlog
			WHEN 0 THEN 'Never'
			WHEN 1 THEN 'When the job succeeds'
			WHEN 2 THEN 'When the job fails'
			WHEN 3 THEN 'When the job completes'
		END 
		, [Notify_Level_Email] =
		CASE msdb.dbo.sysjobs.notify_level_email 
			WHEN 0 THEN 'Never'
			WHEN 1 THEN 'When the job succeeds'
			WHEN 2 THEN 'When the job fails'
			WHEN 3 THEN 'When the job completes'
		END 
		, [OnFailureNotify] = CASE WHEN msdb.dbo.sysjobs.notify_level_email = 2 THEN msdb.dbo.sysoperators.name ELSE '' END
		, [OperatorEmail] = msdb.dbo.sysoperators.email_address
		, [Description] = msdb.dbo.sysjobs.[description]
		, [OutputFile] = CASE WHEN l.LogCount != 1 THEN 'Check Logs'
								ELSE l.LogPath
							END
FROM	msdb.dbo.sysjobs
		LEFT OUTER JOIN msdb.dbo.sysjobschedules ON msdb.dbo.sysjobs.job_id = msdb.dbo.sysjobschedules.job_id
		LEFT OUTER JOIN  msdb.dbo.sysschedules
			ON msdb.dbo.sysjobschedules.schedule_id = msdb.dbo.sysschedules.schedule_id
			AND msdb.dbo.sysschedules.[enabled] = ISNULL(@Enabled,msdb.dbo.sysschedules.[enabled])
		LEFT OUTER JOIN msdb.dbo.sysoperators ON msdb.dbo.sysjobs.notify_email_operator_id = msdb.dbo.sysoperators.id
		LEFT OUTER JOIN (
			SELECT  job_id
					,StepCount = COUNT(*)
					,LogCount = COUNT(DISTINCT output_file_name)
					,LogPath = MAX(output_file_name)
			FROM    msdb.dbo.sysjobsteps
			--WHERE	subsystem = 'TSQL'
			GROUP BY job_id
		) l ON l.job_id = sysjobs.job_id
WHERE	msdb.dbo.sysjobs.date_modified >= ISNULL(@LastModDate,msdb.dbo.sysjobs.date_modified)
ORDER BY msdb.dbo.sysjobs.name
		, msdb.dbo.sysschedules.freq_type


Get SQL Agent Job Steps

In [ ]:
SET NOCOUNT ON;

SELECT @@SERVERNAME AS servername 
	,j.name AS jobname 
	,js.step_id 
	,js.step_name 
	,js.subsystem 
	,js.command 
	,js.on_success_action 
	,js.on_success_step_id 
	,js.on_fail_action 
	,js.on_fail_step_id 
	,js.database_name 
	,js.database_user_name 
	,js.retry_attempts 
	,js.retry_interval 
	,js.output_file_name
	,CASE WHEN j.start_step_id = js.step_id THEN 1 ELSE 0 END AS start_step_id
FROM msdb..sysjobs j
	INNER JOIN msdb..sysjobsteps js ON j.job_id = js.job_id;



Get SQL Agent Job Schedules

In [ ]:
SET NOCOUNT ON;

SELECT @@SERVERNAME AS servername 
	,j.name AS jobname 
	,s.name AS schedulename 
	,s.enabled 
	,s.freq_type 
	,s.freq_interval 
	,s.freq_subday_type 
	,s.freq_subday_interval 
	,s.freq_relative_interval 
	,s.freq_recurrence_factor 
	,s.active_start_date 
	,s.active_start_time 
	,s.active_end_time
FROM msdb..sysjobs j
	INNER JOIN msdb..sysjobschedules js ON j.job_id = js.job_id
	INNER JOIN msdb..sysschedules s ON js.schedule_id = s.schedule_id;


Get Availability Group Detail

In [ ]:
SET NOCOUNT ON;

-- RUN ON PRIMARY --

DECLARE @IsHadrEnabled TINYINT = CONVERT(TINYINT,SERVERPROPERTY ('IsHadrEnabled'))
		,@ServerName NVARCHAR(256) = @@SERVERNAME 
		,@RoleDesc NVARCHAR(60) = 'PRIMARY'
		,@AG_SQL NVARCHAR(MAX)
		,@Version VARCHAR(50) = LEFT(CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR(128)),2);

DECLARE @AGs TABLE (
	domain VARCHAR(128) NOT NULL
	,availabilitygroupname VARCHAR(128) NOT NULL
	,replicaservername VARCHAR(128) NOT NULL
	,endpoint_url VARCHAR(128) NULL
	,availability_mode TINYINT NOT NULL
	,failover_mode TINYINT NOT NULL
	,session_timeout INT NOT NULL
	,primary_role_allow_connections TINYINT NOT NULL
	,secondary_role_allow_connections TINYINT NOT NULL
	,backup_priority INT NOT NULL
	,read_only_routing_url VARCHAR(256) NULL
	,seeding_mode TINYINT NULL
	,read_only_routing_lists VARCHAR(1000) NULL
);

IF (@IsHadrEnabled = 1)
AND EXISTS (
	SELECT 1
	FROM sys.dm_hadr_availability_replica_states AS a
		JOIN sys.availability_replicas AS b
	ON b.replica_id = a.replica_id
	WHERE b.replica_server_name = @ServerName
	AND	a.role_desc = @RoleDesc
)
BEGIN
	SELECT *
	INTO #availability_read_only_routing_lists
	FROM sys.availability_read_only_routing_lists;

	SELECT *
	INTO #availability_replicas
	FROM sys.availability_replicas;
	
	SELECT group_id, name
	INTO #availability_groups
	FROM sys.availability_groups;

	SET @AG_SQL = '
	WITH read_only_routing_lists AS (
	
		SELECT	r.replica_id
				,STUFF((
					SELECT '','' + ror.replica_server_name
					FROM #availability_read_only_routing_lists l
						JOIN #availability_replicas ror ON l.read_only_replica_id = ror.replica_id
					WHERE r.replica_id = l.replica_id
					ORDER BY l.routing_priority
					FOR XML PATH(''''),TYPE).value(''.'',''VARCHAR(MAX)''),1,1,'''') AS read_only_replicas
		FROM	#availability_replicas r
	)

SELECT	DEFAULT_DOMAIN() as domain
		,ag.name
		,r.replica_server_name
		,r.endpoint_url
		,r.availability_mode
		,r.failover_mode
		,r.session_timeout
		,r.primary_role_allow_connections
		,r.secondary_role_allow_connections
		,r.backup_priority
		,r.read_only_routing_url'
		+ CASE WHEN @Version > 11 THEN '	,r.seeding_mode' ELSE '	,NULL' END + '
		,ro.read_only_replicas
FROM	#availability_groups ag
		JOIN #availability_replicas r on ag.group_id = r.group_id
		LEFT JOIN read_only_routing_lists ro ON r.replica_id = ro.replica_id'

	INSERT @AGs (
		domain
		,availabilitygroupname
		,replicaservername
		,endpoint_url
		,availability_mode
		,failover_mode
		,session_timeout
		,primary_role_allow_connections
		,secondary_role_allow_connections
		,backup_priority
		,read_only_routing_url
		,seeding_mode
		,read_only_routing_lists
	)
	EXEC sp_executesql @AG_SQL;
END;

SELECT * FROM @AGs;

IF OBJECT_ID('tempdb..#availability_read_only_routing_lists') IS NOT NULL
 DROP TABLE #availability_read_only_routing_lists;

IF OBJECT_ID('tempdb..#availability_replicas') IS NOT NULL
 DROP TABLE #availability_replicas;
 
IF OBJECT_ID('tempdb..#availability_groups') IS NOT NULL
 DROP TABLE #availability_groups;


Get Availability Group Replica Detail

In [ ]:
SET NOCOUNT ON; 

-- RUN ON PRIMARY --

DECLARE @IsHadrEnabled TINYINT = CONVERT(TINYINT,SERVERPROPERTY ('IsHadrEnabled'))
		,@ServerName NVARCHAR(256) = @@SERVERNAME 
		,@RoleDesc NVARCHAR(60) = 'PRIMARY'
		,@AG_SQL NVARCHAR(MAX)
		,@Version VARCHAR(50) = LEFT(CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR(128)),2);

DECLARE @AGs TABLE (
	domain VARCHAR(128) NOT NULL
	,primaryreplicaservername VARCHAR(128) NOT NULL
	,availabilitygroupname VARCHAR(128) NOT NULL
	,failure_condition_level INT NOT NULL
	,health_check_timeout INT NOT NULL
	,automated_backup_preference TINYINT NOT NULL
	,listener_name VARCHAR(63) NULL
	,listener_port INT NULL
	,listener_ip VARCHAR(200) NULL
	,dtc_support BIT NULL
	,db_failover BIT NULL
);

IF (@IsHadrEnabled = 1)
AND EXISTS (
	SELECT 1
	FROM sys.dm_hadr_availability_replica_states AS a
		JOIN sys.availability_replicas AS b
	ON b.replica_id = a.replica_id
	WHERE b.replica_server_name = @ServerName
	AND	a.role_desc = @RoleDesc
)
BEGIN
	SELECT *
	INTO #availability_groups
	FROM sys.availability_groups;

	SELECT group_id, dns_name, port, ip_configuration_string_from_cluster
	INTO #availability_group_listeners
	FROM sys.availability_group_listeners;

	SET @AG_SQL = '
	SELECT	DEFAULT_DOMAIN() as domain
			,''' + @ServerName + ''' as primaryreplicaservername
			,ag.name
			,ag.failure_condition_level
			,ag.health_check_timeout
			,ag.automated_backup_preference
			,l.dns_name
			,l.port
			,l.ip_configuration_string_from_cluster'
	+ CASE WHEN @Version > 11 THEN '	,ag.dtc_support' ELSE '	,NULL' END
	+ CASE WHEN @Version > 11 THEN '	,ag.db_failover' ELSE '	,NULL' END + '
	FROM	#availability_groups ag
			JOIN #availability_group_listeners l ON ag.group_id = l.group_id'

	INSERT @AGs (
		domain
		,primaryreplicaservername
		,availabilitygroupname
		,failure_condition_level
		,health_check_timeout
		,automated_backup_preference
		,listener_name
		,listener_port
		,listener_ip
		,dtc_support
		,db_failover
	)
	EXEC sp_executesql @AG_SQL;
END;

SELECT * FROM @AGs;

IF OBJECT_ID('tempdb..#availability_groups') IS NOT NULL
 DROP TABLE #availability_groups;

IF OBJECT_ID('tempdb..#availability_group_listeners') IS NOT NULL
 DROP TABLE #availability_group_listeners;
